## Step 1. Fetch Data form Database

In [1]:
import pandas as pd
import numpy as np
import ta,pymongo,dash,os,sys
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.subplots as sp
import yfinance as yf
# Fetch data from database

# Connect to MongoDB
client = pymongo.MongoClient("localhost", 27017)
# Select the database
db = client["historic_data"]
# Select the collection
collection = db["daily_stock_price"]
# Fetch specific stock
data = collection.find({'symbol': 'AEM'})
# Convert to pandas dataframe
df = pd.DataFrame(list(data))

# Set maximum number of columns to display
pd.set_option('display.max_columns', None)  
# Set maximum width for each column
pd.set_option('display.max_colwidth', None)  
# Set maximum width for the entire DataFrame
pd.set_option('display.width', None)  

In [2]:
# Add project root to sys.path dynamically
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from config.mongdb_config import load_mongo_config

URL = load_mongo_config()['url']

def fetch_data(database, collection, symbol=None,interval=None,datetime=None):
    # Connect to MongoDB
    client = pymongo.MongoClient(URL)
    # Select the database
    db = client[database]
    # Select the collection
    collection = db[collection]
    # Fetch specific stock
    data = collection.find({'symbol': symbol,
                            'interval':interval, 
                            "datetime" : {"$gte":pd.to_datetime(datetime)}}) if symbol else collection.find()
    # Convert to pandas dataframe
    df = pd.DataFrame(list(data))
    return df


## Step 4. Simulated trading

In [5]:
import pandas as pd

class TradingStrategy:
    def __init__(self, df, initial_capital=10000):
        self.df = df
        self.initial_capital = initial_capital
        self.trades = []
        self.current_trade = {}
        self.total_values = initial_capital

    def execute_trades(self):
        for date in range(len(self.df) - 1):
            try:
                # Sell Trade
                if len(self.current_trade) != 0:
                    if (self.df['dual_channel_Alert'].iloc[date] == 0) or date == len(self.df) - 1:
                        # Has Trade and Bearish
                        self.trades.append(
                            {
                                "Entry_price": self.current_trade["entry_price"],
                                "Entry_date": self.current_trade["entry_date"],
                                "Exit_price": self.df['open'].iloc[date],
                                "Exit_date": self.df['date'].iloc[date],
                                "profit": (self.df['open'].iloc[date] / self.current_trade['entry_price']) - 1,
                                "total_asset": (self.total_values * (self.df.iloc[date].open / self.current_trade['entry_price']) - 1)
                            }
                        )
                        # Update total asset
                        self.total_values = (self.total_values * (self.df.iloc[date + 1].open / self.current_trade['entry_price']))

                        # Close Trade
                        self.current_trade = {}

                # Buy Trade
                elif (self.df['dual_channel_Alert'].iloc[date] == 1) and (len(self.current_trade) == 0):  # No Trade and Bullish
                    self.current_trade["entry_price"] = self.df['close'].iloc[date]
                    self.current_trade["entry_date"] = self.df['date'].iloc[date]

            except IndexError:
                print("No Trade Data")

    def get_trades(self):
        return pd.DataFrame(self.trades)

    def get_total_return(self):
        if self.trades:
            return self.trades[-1]['total_asset']
        return 0

# Usage

strategy = TradingStrategy(testing_df)
strategy.execute_trades()
trades_df = strategy.get_trades()
total_return = strategy.get_total_return()

print(f"Technical Strategic Total Return: {total_return}")

total_return = 0

Technical Strategic Total Return: 2903.2886184106187


## Step 5. Dashboard Presentation

In [6]:
df = collection.find()

# Initialize Dash app
app = dash.Dash(__name__)

# Create figure with subplots
fig = sp.make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1, row_heights=[0.65, 0.15, 0.25])

# Define the layout of the Dash app
app.layout = html.Div([
    dcc.Dropdown(
        id='stock-selector',
        options=[{'label': symbol, 'value': symbol} for symbol in collection.distinct("symbol")],
        value=collection.distinct("symbol")[0]
    ),
    dcc.Graph(id='sandbox_testing_graph', figure=fig)
])

# Callback to update the graph based on selected stock and selected range
@app.callback(
    Output('sandbox_testing_graph', 'figure'),
    [Input('stock-selector', 'value'), Input('sandbox_testing_graph', 'relayoutData')]
)
def update_chart(selected_stock, relayout_data):
    # Fetch specific stock
    filtered_query = collection.find({"symbol": f"{selected_stock}"})
    # Convert to pandas dataframe
    filtered_df = pd.DataFrame(list(filtered_query))   
    # Add technical features
    filtered_df = add_features(filtered_df).apply()

    # Add Alerts
    filtered_df = Alert(filtered_df).add_alert()
    
    # Sandbox Testing
    trades_history = TradingStrategy(filtered_df)
    trades_history.execute_trades()
    filtered_trades = trades_history.get_trades()
    
    # Create figure with subplots
    fig = sp.make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1, row_heights=[0.65, 0.15, 0.25])

    # Candlestick chart
    fig.add_trace(go.Candlestick(
        x=filtered_df['date'],
        open=filtered_df['open'],
        high=filtered_df['high'],
        low=filtered_df['low'],
        close=filtered_df['close'],
        name='price'), row=1, col=1)

    # Add EMA traces as lines
    fig.add_trace(go.Scatter(x=filtered_df['date'], y=filtered_df['144EMA'], 
                            mode="lines", name="EMA 144"), row=1, col=1)
    fig.add_trace(go.Scatter(x=filtered_df['date'], y=filtered_df['169EMA'],
                            mode="lines", name="EMA 169"), row=1, col=1)
    fig.add_trace(go.Scatter(x=filtered_df['date'], y=filtered_df['13EMA'],
                            mode="lines", name="EMA 13"), row=1, col=1)
    fig.add_trace(go.Scatter(x=filtered_df['date'], y=filtered_df['8EMA'],
                            mode="lines", name="EMA 8"), row=1, col=1)

    # Add Buy / Sell Annotations
    fig.add_trace(go.Scatter(
                    x=filtered_trades.Entry_date,
                    y=filtered_trades.Entry_price,
                    mode="markers",
                    customdata=filtered_trades,
                    marker_symbol="diamond-dot",
                    marker_size=8,
                    marker_line_width=2,
                    marker_line_color="rgba(0,0,0,0.7)",
                    marker_color="rgba(0,255,0,0.7)",
                    hovertemplate="Entry Time: %{customdata[1]}<br>" +\
                        "Entry Price: %{y:.2f}<br>" +\
                        "Total Asset: %{customdata[5]:.3f}",
                    name="Entries"), row=1, col=1)

    fig.add_trace(go.Scatter(
                    x=filtered_trades.Exit_date,
                    y=filtered_trades.Exit_price,
                    mode="markers",
                    customdata=filtered_trades,
                    marker_symbol="diamond-dot",
                    marker_size=8,
                    marker_line_width=2,
                    marker_line_color="rgba(0,0,0,0.7)",
                    marker_color="rgba(255,0,0,0.7)",
                    hovertemplate="Exit Time: %{customdata[1]}<br>" +\
                        "Exit Price: %{y:.2f}<br>" +\
                        "Total Asset: %{customdata[5]:.3f}",
                    name="Exits"), row=1, col=1)

    # Add MACD subplot
    fig.add_trace(go.Scatter(x=filtered_df['date'], y=filtered_df['MACD'], 
                            mode='lines', name='MACD'), row=2, col=1)
    fig.add_trace(go.Scatter(x=filtered_df['date'], y=filtered_df['MACD_SIGNAL'],
                            mode='lines', name='MACD signal'), row=2, col=1)

    # Add Profit subplot
    fig.add_trace(go.Scatter(x=filtered_trades['Exit_date'], y=filtered_trades['total_asset'], 
                            mode='lines', name='Profit'), row=3, col=1)

    # Layout settings
    fig.update_layout(
        xaxis_rangeslider_visible=False,
        autosize=False,
        width=1000,  # Width of the figure in pixels
        height=800,  # Height of the figure in pixels
    )

    # If a new x-axis range is selected or zoomed
    if relayout_data and 'xaxis.range[0]' in relayout_data and 'xaxis.range[1]' in relayout_data:
        x_start = relayout_data['xaxis.range[0]']
        x_end = relayout_data['xaxis.range[1]']

        # Filter the data based on the selected x-axis range
        filtered_data = filtered_df[(filtered_df['date'] >= x_start) & (filtered_df['date'] <= x_end)]
        trades_filtered = filtered_trades[(filtered_trades['Exit_date'] >= x_start) & (filtered_trades['Exit_date'] <= x_end)]
        
        # Calculate the new y-axis range for the candlestick chart
        y_min = filtered_data['low'].min()
        y_max = filtered_data['high'].max()

        # Calculate the new y-axis range for the MACD chart
        y_min_macd = filtered_data[['MACD', 'MACD_SIGNAL']].min().min()
        y_max_macd = filtered_data[['MACD', 'MACD_SIGNAL']].max().max()

        # Calculate the new y-axis range for the Profit chart
        y_min_profit = trades_filtered['total_asset'].min()
        y_max_profit = trades_filtered['total_asset'].max()

        # Update the figure with new y-axis and x-axis ranges
        fig.update_xaxes(range=[x_start, x_end], row=1, col=1)
        fig.update_xaxes(range=[x_start, x_end], row=2, col=1)  
        fig.update_xaxes(range=[x_start, x_end], row=3, col=1) 
        
        fig.update_yaxes(range=[y_min, y_max], row=1, col=1)
        fig.update_yaxes(range=[y_min_macd, y_max_macd], row=2, col=1)
        fig.update_yaxes(range=[y_min_profit, y_max_profit], row=3, col=1)

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)